# CADP tools queries 

In [109]:
# Choose file 
a = !ls /home/abdelouahab/PFE/lcm-tests/cadp/graph/bcg/*.bcg
e = [print(i) for i in a]

/home/abdelouahab/PFE/lcm-tests/cadp/graph/bcg/9M-5-[1-2001]-[a]-lcm.bcg
/home/abdelouahab/PFE/lcm-tests/cadp/graph/bcg/sample.bcg


In [77]:
file_name = "sample.bcg"
file_path = f"../graph/bcg/{file_name}"

In [106]:
def print_query_content(query_path):
    !cat {query_path}

##   Product adoption query

In [102]:
# Query content 
query_name = "product_adoption.mcl"
query_path = f"../queries/mcl/{query_name}"
print_query_content(query_path)

(* Product Adoption :*)

macro QUERY () =
     {!"G"}.
     {SELF ?C0:String ?P0:String !"T0"}.
     {?any}*.
     {SELF ?C1:String ?P1:String !"T1" where
        (diff (string2natset (P1), string2natset (P0)) <> empty)
     }
end_macro

macro FINAL_STATE () =
    [ not { SELF ... } ] false
end_macro

<
    (true* . QUERY . true*)+
> FINAL_STATE


In [103]:
%%bash -s $file_path $query_path
export PATH=$PATH:$CADP/bin.x64
$CADP/com/bcg_open $1 evaluator4 -diag $2

bcg_open: using ``/home/abdelouahab/cadp//bin.x64/evaluator4.a''
bcg_open: running ``evaluator4 -diag ../queries/mcl/product_adoption.mcl'' for ``../graph/bcg/sample.bcg''

TRUE
*** diagnostic sequence found at depth 5

<initial state>
"G"
"SELF !:1:2:3:4:5:6: !:1:3:4: !T0"
"G"
"SELF !:1:2:3:4:5:6:9: !:1:3:15 !T1"
"ST"
<goal state>


/tmp/caesar_20891_1.c: In function 'caesar_iterate_boolean_variable_0':
/tmp/caesar_20891_1.c:368:12: warning: variable 'caesar_x_variable_0' set but not used [-Wunused-but-set-variable]
  368 | ADT_STRING caesar_x_variable_0 = NULL;
      |            ^~~~~~~~~~~~~~~~~~~
/tmp/caesar_20891_1.c:465:12: warning: variable 'caesar_x_variable_2' set but not used [-Wunused-but-set-variable]
  465 | ADT_STRING caesar_x_variable_2 = NULL;
      |            ^~~~~~~~~~~~~~~~~~~


## Product Loyalty query 

In [104]:
query_path = "../queries/mcl/product_loyalty.mcl"
print_query_content(query_path)

(* Product Loyalty :*)

macro QUERY () =
     {!"G"}.
     {SELF ?C0:String ?P0:String !"T0"}.
     {?any}*.
     {SELF ?C1:String ?P1:String !"T1" where
        (inter (string2natset (P1), string2natset (P0)) <> empty)
     }
end_macro

macro FINAL_STATE () =
    [ not { SELF ... } ] false
end_macro

<
    (true* . QUERY . true*)+
> FINAL_STATE


In [105]:
%%bash -s $file_path $query_path
export PATH=$PATH:$CADP/bin.x64
$CADP/com/bcg_open $1 evaluator4 -diag $2

bcg_open: using ``/home/abdelouahab/cadp//bin.x64/evaluator4.a''
bcg_open: running ``evaluator4 -diag ../queries/mcl/product_loyalty.mcl'' for ``../graph/bcg/sample.bcg''

TRUE
*** diagnostic sequence found at depth 5

<initial state>
"G"
"SELF !:1:2:3:4:5:6: !:1:3:4: !T0"
"G"
"SELF !:1:2:3:4:5:6:9: !:1:3:15 !T1"
"ST"
<goal state>


/tmp/caesar_21249_1.c: In function 'caesar_iterate_boolean_variable_0':
/tmp/caesar_21249_1.c:368:12: warning: variable 'caesar_x_variable_0' set but not used [-Wunused-but-set-variable]
  368 | ADT_STRING caesar_x_variable_0 = NULL;
      |            ^~~~~~~~~~~~~~~~~~~
/tmp/caesar_21249_1.c:465:12: warning: variable 'caesar_x_variable_2' set but not used [-Wunused-but-set-variable]
  465 | ADT_STRING caesar_x_variable_2 = NULL;
      |            ^~~~~~~~~~~~~~~~~~~


## Promotion only

In [113]:
query_path = "../queries/mcl/promotion_only.mcl"
print_query_content(query_path)

(* Promotion only  :*)

macro QUERY () =
     {!"G"}.
     {SELF ?C0:String ?P0:String !"T0"}.
     {?any}*.
     {SELF ?C1:String ?P1:String !"T1"}.
     {?any}*.
     {SELF ?C2:String ?P2:String !"T2" where
             (diff (string2natset (P2), union(string2natset (P0),string2natset (P1))) <> empty)
     }
end_macro

macro FINAL_STATE () =
    [ not { SELF ... } ] false
end_macro

<
    (true* . QUERY . true*)+
> FINAL_STATE


In [114]:
%%bash -s $file_path $query_path
export PATH=$PATH:$CADP/bin.x64
$CADP/com/bcg_open $1 evaluator4 -diag $2

bcg_open: using ``/home/abdelouahab/cadp//bin.x64/evaluator4.a''
bcg_open: running ``evaluator4 -diag ../queries/mcl/promotion_only.mcl'' for ``../graph/bcg/sample.bcg''

TRUE
*** diagnostic sequence found at depth 6

<initial state>
"G"
"SELF !:1:2:3:4:5:6: !:1:3:4: !T0"
"G"
"SELF !:1:2:3:4:5:6:9: !:1:3:15 !T1"
"SG"
"SELF !:7:8:9: !:1:2:15:19: !T2"
<goal state>


/tmp/caesar_22443_1.c: In function 'caesar_iterate_boolean_variable_0':
/tmp/caesar_22443_1.c:373:12: warning: variable 'caesar_x_variable_0' set but not used [-Wunused-but-set-variable]
  373 | ADT_STRING caesar_x_variable_0 = NULL;
      |            ^~~~~~~~~~~~~~~~~~~
/tmp/caesar_22443_1.c:470:12: warning: variable 'caesar_x_variable_2' set but not used [-Wunused-but-set-variable]
  470 | ADT_STRING caesar_x_variable_2 = NULL;
      |            ^~~~~~~~~~~~~~~~~~~
/tmp/caesar_22443_1.c:616:12: warning: variable 'caesar_x_variable_4' set but not used [-Wunused-but-set-variable]
  616 | ADT_STRING caesar_x_variable_4 = NULL;
      |            ^~~~~~~~~~~~~~~~~~~
